In [ ]:
%pip install qiskit
%pip install pylatexenc
%pip install qiskit_aer
%pip install qiskit_ibm_runtime

In [26]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import matplotlib
import qiskit_aer
import pylatexenc
import random
import qiskit
from scipy.optimize import minimize

from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Implementing and benchmarking the 3-qubit surface code

## My best estimate for the pseudocode/components we'll need

#### Create a single logical qubit
      

*   Define a lattice of physical qubits (4x4?)
*   Define the stabilizer generators (vertex and plaquette stabilizers)
*   We're gonna do a rotated surface code -- this is state of the art


#### Replicate the logical qubit twice




#### Figure out how to implement interactions between the qubits



There's several alternate schemes depending on how you construct the lattice and if you want it to encode two logical qubits or one. https://arxiv.org/pdf/quant-ph/9811052 I mainly followed along with paper for two logical qubits and how they implemented the stabilizer circuits.

In [90]:
class DisjointSet:
  def __init__(self, vx_count):
    self.parent = list(range(0,vx_count))
    self.paths = []
    for i in range(0,vx_count):
       self.paths.append([])
    

    self.rank = [0] * vx_count

  def find(self, i):
    if self.parent[i] != i:
      self.parent[i] = self.find(self.parent[i])
    return self.parent[i]

  def union(self, x, y, e):
    s1 = self.find(x)
    s2 = self.find(y)
    #print("Run with "+str(x)+" "+str(y))
    #print(self.rank)
    if s1 != s2:

      if self.rank[s1] < self.rank[s2]:
          self.parent[s1] = s2
          self.rank[s2] += 1 + self.rank[s1]
          self.paths[s1].append(e)
          self.paths[s2] += self.paths[s1]

      else:
          self.parent[s2] = s1
          self.rank[s1] += 1 + self.rank[s2]
          self.paths[s2].append(e)
          self.paths[s1] += self.paths[s2]
    #print(self.rank)
    #print(" ")
  def get_highest_rank_tree(self):
    return self.paths[self.rank.index(max(self.rank))]


In [91]:
class Graph:
  def __init__(self):
    self.vertexes = []
    self.edges = []
    self.minimum_spanning_tree = None
  def add_vertex(self, v):
    self.vertexes.append(v)
  def get_edge_other_vertex(self, v, edge):
    if v == edge["v1"]: return edge["v2"]
    else: return edge["v1"]

  def add_edge(self, v1, v2, w, path):
    self.edges.append({"v1":v1, "v2":v2, "weight":w, "data_qubit_path": path})

  def sort_fun(self, edge):
    return edge["weight"]

  def get_minimum_spanning_tree(self):

    #Sort edges so we go through them highest to lowest order of weight
    self.edges.sort(key=self.sort_fun)
    tree_tracker = DisjointSet(len(self.vertexes))
    count = 0

    for e in self.edges:
      if tree_tracker.find(e["v1"]) != tree_tracker.find(e["v2"]):
        tree_tracker.union(e["v1"], e["v2"], e)
        count += 1
        if count == len(self.vertexes)-1:
          break

    #We've obtained a minimum spanning tree
    min_span = tree_tracker.get_highest_rank_tree()

    #Disconnect the middle edges to get a set of disconnected edges
    min_span.sort(key=self.sort_fun)

    count_edges_vertex_in = dict((x,0) for x in self.vertexes)
    vertex_to_edges = dict((x,[]) for x in self.vertexes)

    for e in min_span:
      count_edges_vertex_in[e["v1"]] += 1
      count_edges_vertex_in[e["v2"]] += 1
      vertex_to_edges[e["v1"]].append(e)
      vertex_to_edges[e["v2"]].append(e)

    print(count_edges_vertex_in)
    #only_one = list(x for x in self.vertexes if count_edges_vertex_in[x]==1)
    count_edges_vertex_in = dict(sorted(filter(lambda item: item[1], count_edges_vertex_in.items()), key=lambda item: item[1]))
    
    vertex_used = []
    shortest_edges = []

    print(count_edges_vertex_in)
    #print(vertex_to_edges)
    for v in count_edges_vertex_in:
      print(v)
      print(vertex_used)
      while v not in vertex_used:
  
        for e in vertex_to_edges[v]:
          if self.get_edge_other_vertex(v,e) not in vertex_used:
            vertex_used.append(v)
            vertex_used.append(self.get_edge_other_vertex(v,e))
            shortest_edges.append(e)

      if len(vertex_used) == len(self.vertexes):
        break


    return shortest_edges





In [92]:
#Graph Tests
test_graph = Graph()
#Create a graph of stabilizers with errors on 1 and 10th qubit
for i in range(0,9):
  test_graph.add_vertex(i)



test_graph.add_edge(1,2,1,[1])
test_graph.add_edge(1,4,1,[4])
test_graph.add_edge(1,7,2,[4,10])

test_graph.add_edge(4,7,1,[10])
test_graph.add_edge(2,4,2,[1,4])
test_graph.add_edge(2,7,3,[1,4,10])

print(test_graph.vertexes)
print(test_graph.edges)

min_tree = test_graph.get_minimum_spanning_tree()
print(min_tree)




[0, 1, 2, 3, 4, 5, 6, 7, 8]
[{'v1': 1, 'v2': 2, 'weight': 1, 'data_qubit_path': [1]}, {'v1': 1, 'v2': 4, 'weight': 1, 'data_qubit_path': [4]}, {'v1': 1, 'v2': 7, 'weight': 2, 'data_qubit_path': [4, 10]}, {'v1': 4, 'v2': 7, 'weight': 1, 'data_qubit_path': [10]}, {'v1': 2, 'v2': 4, 'weight': 2, 'data_qubit_path': [1, 4]}, {'v1': 2, 'v2': 7, 'weight': 3, 'data_qubit_path': [1, 4, 10]}]
{0: 0, 1: 2, 2: 1, 3: 0, 4: 2, 5: 0, 6: 0, 7: 1, 8: 0}
{2: 1, 7: 1, 1: 2, 4: 2}
2
[]
7
[2, 1]
1
[2, 1, 7, 4]
4
[2, 1, 7, 4]
[{'v1': 1, 'v2': 2, 'weight': 1, 'data_qubit_path': [1]}, {'v1': 4, 'v2': 7, 'weight': 1, 'data_qubit_path': [10]}]


In [81]:
min_tree = test_graph.get_minimum_spanning_tree()
print(min_tree)

Run with 1 2
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 3, 1, 1, 1, 1, 1, 1, 1]
 
Run with 1 4
[1, 3, 1, 1, 1, 1, 1, 1, 1]
[1, 5, 1, 1, 1, 1, 1, 1, 1]
 
Run with 4 7
[1, 5, 1, 1, 1, 1, 1, 1, 1]
[1, 7, 1, 1, 1, 1, 1, 1, 1]
 
{0: 0, 1: 2, 2: 1, 3: 0, 4: 2, 5: 0, 6: 0, 7: 1, 8: 0}
{0: 0, 3: 0, 5: 0, 6: 0, 8: 0, 2: 1, 7: 1, 1: 2, 4: 2}
0
[]


KeyboardInterrupt: 

In [ ]:
test_graph_2 = Graph()
for i in range(0,9):
    test_graph_2.add_vertex(i)



In [4]:
class Node:
    def __init__(self, parent, name):
      self.parent = parent
      self.name = name
      self.children = []

    def add_child(self, c):
      self.children.append(c)

    def add_children(self, c):
      self.children = self.children + c


In [ ]:
#Test Node
test_node = Node(None, "p")
test_child1 =Node(test_node, "c1")
test_child2 =Node(test_node, "c2")
test_child3 =Node(test_node, "c3")

children = [test_child2,test_child3]

test_node.add_child(test_child1)
print(test_node.children)

test_node.add_children(children)
print(test_node.children)


In [ ]:
class Tree:
  def __init__(self, root):
    self.root = Node(None, root)
    self.nodes = {root: self.root}
  def add_to_path(self, parent, node):
    p = self.nodes[parent]

    if isinstance(node, list):
      for n in node:
        self.nodes[n] = Node(p, n)
        p.add_child(self.nodes[n])
    else:
      self.nodes[node] = Node(p, node)
      p.add_child(self.nodes[node])

  def get_path_to_root(self, node):
    path = []
    distance = 0
    n = self.nodes[node]
    while n != None:
      path = n.name + path
      distance = distance + 1
      n = n.parent

    return path, distance-1




In [ ]:
# Test Tree
#Initalized successfully
test_tree = Tree(1)
print(test_tree.root)
print(test_tree.nodes)

#Test adding a single one
test_tree.add_to_path(1,2)
print(test_tree.nodes)

#Test multi add
test_tree.add_to_path(1,[3,4])
print(test_tree.nodes)

test_tree.add_to_path(2,5)
print(test_tree.nodes)

path, d = test_tree.get_path_to_root(5)
print(path)
print(d)

path, d = test_tree.get_path_to_root(3)
print(path)
print(d)


In [10]:
#TODO: Add Description
class SurfaceCode:

  def __init__(self, d, T):
    '''
    Distance d surface code for T rounds of error correction (syndrome measurement)
    * A surface code of distance d needs 2*d**2 physical qubits to support it the encodes the logical qubit value
    *
    '''
    self.d = d
    self.T = 0
    #TODO: Edit to give correct (2d-1)^2
    # SHREYA THOUGHTS: no this is not true -- d is the distance of the code = distance to the next valid state (qubit configuration). a code can correct floor((d-1)/2) errors
    # TODO: what is n? number of physical qubits in the lattice? afaik that is (2d-1)^2 for an unrotated surface code. for toric code we have [n,k,d] = [(2d)^2, 2, d]
    # for example we are implementing a [25, 1, 3] surface code if we're going for unrotated (this is the smallest one possible), so n = (6-1)^2 = 25
    self.n = d - 1

    #qubits
    self.data_qubits = QuantumRegister(2*(self.n*self.n), "data_qubits")
    self.x_stabilizer_qubits = QuantumRegister(self.n*self.n, "x_stabilizer_qubits")
    self.z_stabilizer_qubits = QuantumRegister(self.n*self.n, "z_stabilizer_qubits")
    #classical
    self.classical_data_bits = ClassicalRegister(2*(self.n*self.n), "classical_data_bits")
    self.x_syndrome_bits = ClassicalRegister((self.n*self.n), "x_syndrome")
    self.z_syndrome_bits = ClassicalRegister((self.n*self.n), "z_syndrome")
    self.x_old_syndrome = [0] * self.n * self.n
    self.z_old_syndrome = [0] * self.n * self.n
    self.circuit = QuantumCircuit(self.data_qubits, self.x_stabilizer_qubits, self.z_stabilizer_qubits, self.classical_data_bits, self.x_syndrome_bits, self.z_syndrome_bits)
    self.registers = {"data_qubits", "x_stabilizer_qubits","z_stabilizer_qubits","classical_data_bits", "x_syndrome_bits", "z_syndrome_bits"}


    self.lattice = self.construct_lattice(self.n)
    self.x_pauli_frame = [1]*self.n*(self.n*2)
    self.z_pauli_frame = [1]*self.n*(self.n*2)
    self.x_generators = self.create_x_generators(self.lattice,self.n)
    self.z_generators = self.create_z_generators(self.lattice,self.n)


In [13]:


  def construct_lattice(self, n):
    """Constructs a lattice out of qubit indexes.
    Each qubit is an edge - 2-D array where first row is first horionztal qubit row, second is first vertical qubit row"""
    edges = []#np.array()
    horizontal_edges = []
    vertical_edges = []
    for e in range(0,2*n):
      #Horizontal if even, verical if odd
      start = n*e
      row = [*range(start,start+n)]
      edges.append(row)

      '''if e%2 == 0:
          horizontal_edges.append(row)
      else:
        vertical_edges.append(row)'''

    return edges

  def create_x_generators(self, lattice, n):
    '''Creates list of x genarator. X generator is a list of data qubit indexes the generator impacts.'''
    x_gen = []

    for i in range(0,n):
      for j in range(0,n):
        upper_edge = 2*i-1 if 2*i-1 > 0 else 2*n-1

        x_gen.append([lattice[2*i][j],lattice[2*i][(j+1)%n],lattice[upper_edge][j],lattice[2*i+1][j]])

    return x_gen

  def create_z_generators(self, lattice, n):
    '''Creates list of z genarator. Z generator is a list of data qubit indexes the generator impacts.'''
    z_gen = []
    for i in range(0,n):
      for j in range(0,n):
        left_edge = j-1 if j-1 > 0 else n-1
        z_gen.append([lattice[2*i][j]
                      ,lattice[(2*i+2)%n][j]
                      ,lattice[2*i+1][j]
                      ,lattice[2*i+1][left_edge]])
    return z_gen


  def encode_physical_qubits(self, logical_qubit):
    #Logical qubit Encoding
    #TODO: Encode even parity for 0, odd for 1
    #Create superposition of all possible states where rules are followed - I think this
    #TODO Got lost in the variable sauce and wasn't sure the exact translation into a circuit
    self.circuit

  def add_x_stabilizer_measurement(self, stabilizer_index, stabilizer):
    '''Add a specified stabilizer to the surface circuit'''
    self.circuit.h(self.x_stabilizer_qubits[stabilizer_index])
    for q in stabilizer:
      self.circuit.cx(self.x_stabilizer_qubits[stabilizer_index], self.data_qubits[q])

    self.circuit.h(self.x_stabilizer_qubits[stabilizer_index])
    self.circuit.measure(self.x_stabilizer_qubits[stabilizer_index], self.x_syndrome_bits[stabilizer_index])
    self.circuit.barrier()

  def add_z_stabilizer_measurement(self, stabilizer_index, stabilizer):
    for q in stabilizer:
      self.circuit.cx(self.data_qubits[q],self.z_stabilizer_qubits[stabilizer_index])

    self.circuit.measure(self.z_stabilizer_qubits[stabilizer_index], self.z_syndrome_bits[stabilizer_index])
    self.circuit.barrier()
  def reset_circuit(self):
    n = self.d - 1

    #self.x_stabilizer_qubits = QuantumRegister(n*n, "x_stabilizer_qubits")
    #self.z_stabilizer_qubits = QuantumRegister(n*n, "z_stabilizer_qubits")
    #self.classical_data_bits = ClassicalRegister(2*(n*n), "classical_data_bits")
    #self.x_syndrome_bits = ClassicalRegister((n*n), "x_syndrome")
    #self.z_syndrome_bits = ClassicalRegister((n*n), "z_syndrome")
    #self.x_old_syndrome_bits = ClassicalRegister((n*n), "x_old_syndrome")
    #self.z_old_syndrome_bits = ClassicalRegister((n*n), "z_old_syndrome")

    self.circuit = QuantumCircuit(self.data_qubits, self.x_stabilizer_qubits, self.z_stabilizer_qubits, self.classical_data_bits, self.x_syndrome_bits, self.z_syndrome_bits)
    #self.registers = {"data_qubits", "x_stabilizer_qubits","z_stabilizer_qubits","classical_data_bits", "x_syndrome", "z_syndrome"}

  def add_cycle_stabilizers(self):
    "Cycles through stabilizers and updates syndrome qubit measurements. Returns list of x syndrome flags and z syndrome flag and a boolean saying if any flags have happened."
    #add stabilizer
    for i in range(0,self.n*self.n):
      self.add_x_stabilizer_measurement(i,self.x_generators[i])
      self.add_z_stabilizer_measurement(i,self.z_generators[i])

  def error_detection(self, backend, pm, simulator = True):
    "Run circuit and flags syndrome bits based expected parity."
    error_exists = False
    x_flags = [0]*self.n*self.n
    z_flags = [0]*self.n*self.n
    print(self.x_syndrome_bits.size)
    #run circuit
    if simulator:
      result = AerSimulator().run(self.circuit).result()
    else:
      sampler = Sampler(backend)
      transpiled_circuit = pm.run(self.circuit)
      # collect stats
      job = sampler.run([transpiled_circuit])
      result = job.result()
    stats = result.get_counts()
    max = 0
    for s in stats:
      if max<stats[s]:
        max = stats[s]
        syn_string = s

    print(syn_string)
    #print(result)
    syn = syn_string.split()
    #TODO Check ordering first
    x_syndrome = list(map(int, list(syn[1])))
    z_syndrome = list(map(int, list(syn[0])))
    #print(x_syndrome)
    #flag errors
    for i in range(0,self.n*self.n):
      if abs(x_syndrome[i]-self.x_old_syndrome[i]) > 0:
        x_flags[i] = 1
        error_exists = True
      if abs(z_syndrome[i]-self.z_old_syndrome[i]) > 0:
        z_flags[i] = 1
        error_exists = True

    self.x_old_syndrome = x_syndrome.copy()
    self.z_old_syndrome = z_syndrome.copy()

    return x_flags, z_flags, error_exists, result


  def fix_error(self, type, target):
    "To fix or apply an x error."
    if type == "z":
      self.circuit.x(target)
    else:
      self.circuit.z(target)



  def induce_error(self, prob_x_error, prob_z_error):
    "Adds error to the physical bits after matrix is contructed"
    for q in self.data_qubits:
      e = random.random()
      if e < prob_x_error:
        self.circuit.x(q)
      e = random.random()
      if e < prob_z_error:
        self.circuit.z(q)
  def update_pauli_frame(self, x_flagged_qubits, z_flagged_qubits):
    for x in x_flagged_qubits:
      self.z_pauli_frame[x] = self.z_pauli_frame[x]*-1
    for z in z_flagged_qubits:
      self.x_pauli_frame[z] = self.x_pauli_frame[z]*-1

  def apply_pauli_frame(self):
    for i in range(0,len(self.x_pauli_frame)):
      if self.x_pauli_frame[i]<0:
        self.circuit.x(self.data_qubits[i])
      if self.z_pauli_frame(i)<0:
        self.circuit.x(self.data_qubits[i])

  #functions related to the decoding and error correcting process
  def decoder_minimum_weight_perfect_matching(self, x_flags, z_flags):

      x_graph, x_min_distance = self.flag_graphs(x_flags)
      z_graph, z_min_distance = self.flag_graphs(z_flags)

      x_shortest = []
      z_shortest = []
      for edge in x_graph.edges:
        x = 0
        #TODO: Optimization problem set up - CPLEX by ibm
        #if edge["weight"] == x_min_distance:
        #  x_shortest.append(self.data_qubits_transversed_by_path("x", edge["path"]))

      for edge in z_graph.edges:
        y = 0
        #if edge["weight"] == z_min_distance:
        # z_shortest.append(self.data_qubits_transversed_by_path("z", edge["path"]))

  def data_qubits_transversed_by_path(self, type, path):
      recovery_data = []

      if type == "z":
        for i in range(0,len(path)-1):
          overlap = list(set(self.z_generators[path[i]]) & set(self.z_generators[path[i+1]]))
          recovery_data.append(overlap[0])
      else:
        for i in range(0,len(path)-1):
          overlap = list(set(self.x_generators[path[i]]) & set(self.x_generators[path[i+1]]))
          recovery_data.append(overlap[0])

      return recovery_data

  def flag_graphs(self, flags):
      "Creates a graph between flags with the edges weights being determined by the shortest manhattan distance between them"
      min_distance = self.n*self.n
      flag_graph = Graph()
      for k in range(0, len(flags)):
        flag_graph.add_vertex(k)
        #creates shortest path from new vertex to all other points
        for v in flag_graph.vertexes:
          paths = Tree(k)
          if v != k:
            distance = 0
            i1 = k/self.n
            j1 = self.n%k

            i2 = v/self.n
            j2 = self.n%v


            nodesToExplore = [k]

            #Breadth First Search to find shortest path and save searched space as a Tree
            while i != i2 and j != j2:
              i = nodesToExplore[0]/self.n
              j = self.n%nodesToExplore[0]

              childUp = i - 1
              childLeft = j-1
              childDown = i + 1
              childRight = j+1

              paths.add_to_path(nodesToExplore[0],[self.n*childUp+j, self.n*childDown+j,self.n*i+childLeft,self.n*i+childRight])
              nodesToExplore = nodesToExplore + [self.n*childUp+j, self.n*childDown+j,self.n*i+childLeft,self.n*i+childRight]
              nodesToExplore.pop(0)


            path, distance = paths.get_path_to_root(self.n*i+j)
            if distance < min_distance:
              min_distance = distance
            flag_graph.add_edge(k, v, distance,path)
          del paths

      return flag_graph, min_distance

  def recovery_operation(self ):
      g = 0



# Main

In [ ]:
service = QiskitRuntimeService(token='',channel='ibm_quantum')
backend = service.least_busy(simulator = False, operational = True)
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

<ipython-input-21-105c25296384>:1: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(token='e1816d36734769cead0ac8774d406072977fe419aad0d46927507d83a3f2741bd766526b436a3aada0b45242d3a0e7757a789b59ee123d27814f55d3a83c3b6c',channel='ibm_quantum')


In [ ]:
#Create physical bit for state |00>

#induce error

#run first round of stabilizers

#if error detected, record in pauli frame

#repeat for several rounds

#perform pauli frame corrections

#measure data qubits and translate back into logical operation

In [7]:
#Test lattice construction
test = SurfaceCode(4,6)
#Build Lattice
n = 2
lattice= test.construct_lattice(n)
print(2*(n*(n+1))+1)
expected1 = [[0, 1], [2, 3], [4, 5], [6, 7]]
if lattice != expected1:
  print("Mistake! You fool.")
  print("Expected: ")
  print(expected1)
  print("Generated: ")
  print(lattice)

n = 3
lattice= test.construct_lattice(n)
print(2*(n*(n+1))+1)
expected2 = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11], [12, 13, 14], [15, 16, 17]]
if lattice != expected2:
  print("Mistake! You fool.")
  print("Expected: ")
  print(expected2)
  print("Generated: ")
  print(lattice)

13
25


In [ ]:
#Check x and z generator creation
print(test.create_x_generators(lattice,3))

[[0, 1, 15, 3], [1, 2, 16, 4], [2, 0, 17, 5], [6, 7, 3, 9], [7, 8, 4, 10], [8, 6, 5, 11], [12, 13, 9, 15], [13, 14, 10, 16], [14, 12, 11, 17]]


In [ ]:
print(test.create_z_generators(expected2,3))

[[0, 6, 3, 5], [1, 7, 4, 5], [2, 8, 5, 4], [6, 3, 9, 11], [7, 4, 10, 11], [8, 5, 11, 10], [12, 0, 15, 17], [13, 1, 16, 17], [14, 2, 17, 16]]


Stabilizer Tests

In [ ]:
test = SurfaceCode(4,6)
#test.circuit.draw(output="mpl")

In [ ]:
test.add_x_stabilizer_measurement(1,test.x_generators[1])

In [ ]:
#test.circuit.draw(output="mpl")

In [ ]:
test = SurfaceCode(4,6)
del test

# Testing Zone

Messy area where I test for expected behavior and debug simulation. Left in for

In [ ]:
#test.circuit.draw(output="mpl")
#test.add_z_stabilizer_measurement(1,test.z_generators[1])
#test.circuit.draw(output="mpl")

In [ ]:
test = SurfaceCode(4,6)

In [ ]:
#Test registers keep values after circuit cleared - failed so just going to rerun full circuit and do not reset it unless want to start over
test.circuit.x(test.data_qubits[3])
test.circuit.h(test.data_qubits[2])
test.circuit.z(test.data_qubits[2])
test.circuit.h(test.data_qubits[2])
test.circuit.measure(3, test.x_syndrome_bits[2])
test.circuit.measure(2, test.x_syndrome_bits[3])

result = AerSimulator().run(test.circuit).result()

In [ ]:
print(result.get_counts())

{'000000000 000001100 000000000000000000': 1024}


In [ ]:

#test.circuit = QuantumCircuit(test.data_qubits, test.x_stabilizer_qubits, test.z_stabilizer_qubits, test.classical_data_bits, test.x_syndrome_bits, test.z_syndrome_bits)
test.circuit.measure(0, test.x_syndrome_bits[5])
result = AerSimulator().run(test.circuit).result()

In [ ]:
print(result.get_counts())

{'000000000 000001100 000000000000000000': 1024}


In [ ]:
def add_x_stabilizer_measurement(self, stabilizer_index, stabilizer):
    '''Add a specified stabilizer to the surface circuit'''
    test.circuit.h(test.x_stabilizer_qubits[stabilizer_index])
    for q in stabilizer:
      test.circuit.cx(test.x_stabilizer_qubits[stabilizer_index], test.data_qubits[q])

    test.circuit.h(test.x_stabilizer_qubits[stabilizer_index])
    test.circuit.measure(test.x_stabilizer_qubits[stabilizer_index], test.x_syndrome_bits[stabilizer_index])
    test.circuit.barrier()

In [ ]:
test = SurfaceCode(4,6)
test.error_detection(None, None, simulator = True)
#test.circuit.h(test.data_qubits[0])
#test.circuit.x(test.data_qubits[0])
#test.add_z_stabilizer_measurement(0, test.z_generators[0])
#test.circuit.x(test.z_syndrome_bits[0])
'''test.circuit.x(test.data_qubits[0])
test.circuit.h(test.x_stabilizer_qubits[0])
print(test.x_generators[0])
gen = test.x_generators[0]
test.circuit.cx(test.data_qubits[gen[0]],test.x_stabilizer_qubits[0])
test.circuit.h(test.x_stabilizer_qubits[0])
test.add_x_stabilizer_measurement(1,test.x_generators[1])
for i in range(0,4):
  test.circuit.measure(test.z_stabilizer_qubits[i], test.z_syndrome_bits[i])
  test.circuit.measure(test.x_stabilizer_qubits[i], test.x_syndrome_bits[i])

print(test.z_generators[0])
#test.circuit.x(test.data_qubits[0])
#test.circuit.z(test.data_qubits[2])
#test.add_cycle_stabilizers()
#test.circuit.measure(test.x_stabilizer_qubits, test.x_syndrome_bits)
test.circuit.measure(test.data_qubits,test.classical_data_bits)
test.circuit.draw(output="mpl")'''


9


QiskitError: 'No counts for experiment "0"'

In [ ]:

x, z, error, result = test.error_detection(None, None, True)
c = list(result.get_counts().items())[:10]
display(plot_histogram(dict(c)))
if error:
  print(x)
  print(z)

In [ ]:
#Test DisjointSet
v = 6
edgesComplete = []
for i in range(0,v):
    for j in range(i+1,v):
        edgesComplete.append([i,j])

dj = DisjointSet(6)
print(" ")
print("Test 1: No Repeat Union")
print(" ")
#Union
print(dj.parent)
print(dj.paths)
print(dj.rank)
dj.union(0,1,[0,1])
print(" ")
#Check that same edge can't be added twice
dj.union(0,1,[0,1])
print(dj.parent)
print(dj.paths)
print(dj.rank)

print(" ")
print("Test 2: Multi Union")
print(" ")

print(dj.parent)
print(dj.paths)
print(dj.rank)
dj.union(0,1,{"v1":0,"v2":1})
print(" ")
#Check that same edge can't be added twice
dj.union(2,3,[2,3])
print(" ")
dj.union(4,5,[4,5])
print(" ")

print(dj.parent)
print(dj.paths)
print(dj.rank)
print(" ")
dj.union(1,3,[3,3])
print(" ")
print(dj.parent)
print(dj.paths)
print(dj.rank)

 
Test 1: No Repeat Union
 
[0, 1, 2, 3, 4, 5]
[[], [], [], [], [], []]
[1, 1, 1, 1, 1, 1]
Run with 0 1
[1, 1, 1, 1, 1, 1]
[3, 1, 1, 1, 1, 1]
 
 
Run with 0 1
[3, 1, 1, 1, 1, 1]
[3, 1, 1, 1, 1, 1]
 
[0, 0, 2, 3, 4, 5]
[[[0, 1]], [[0, 1]], [], [], [], []]
[3, 1, 1, 1, 1, 1]
 
Test 2: Multi Union
 
[0, 0, 2, 3, 4, 5]
[[[0, 1]], [[0, 1]], [], [], [], []]
[3, 1, 1, 1, 1, 1]
Run with 0 1
[3, 1, 1, 1, 1, 1]
[3, 1, 1, 1, 1, 1]
 
 
Run with 2 3
[3, 1, 1, 1, 1, 1]
[3, 1, 3, 1, 1, 1]
 
 
Run with 4 5
[3, 1, 3, 1, 1, 1]
[3, 1, 3, 1, 3, 1]
 
 
[0, 0, 2, 2, 4, 4]
[[[0, 1]], [[0, 1]], [[2, 3]], [[2, 3]], [[4, 5]], [[4, 5]]]
[3, 1, 3, 1, 3, 1]
 
Run with 1 3
[3, 1, 3, 1, 3, 1]
[7, 1, 3, 1, 3, 1]
 
 
[0, 0, 0, 2, 4, 4]
[[[0, 1], [2, 3], [3, 3]], [[0, 1]], [[2, 3], [3, 3]], [[2, 3]], [[4, 5]], [[4, 5]]]
[7, 1, 3, 1, 3, 1]


In [ ]:
test.circuit.draw(output="mpl")

In [ ]:
#Test that encoder gets circuit set up properly

In [ ]:
#Construct cicuit and starting state
#Induce errors
#Run error correcting scheme
#Test on simulator



In [ ]:
#Run on IBM computers
#Test three different starting states

In [ ]:
#Results vsiualization

Resources used:
* https://arthurpesah.me/blog/2023-05-13-surface-code/
* https://errorcorrectionzoo.org/c/rotated_surface
* https://errorcorrectionzoo.org/c/stab_4_2_2
* https://github.com/quantumjim/qec_lectures/blob/main/lecture-1.ipynb
* https://www.youtube.com/watch?v=M25fBmF9XR0
* https://github.com/The-Singularity-Research/QISKit-Surface-Codes?tab=readme-ov-file

This last link is very reader friendly and straightforward.
* https://decodoku.blogspot.com/2016/03/8-toric-code-part-3.html

Original Paper
* https://arxiv.org/pdf/quant-ph/9811052

Really good paper on nitty gritty on measuring stabilizer qubits
* https://pure.tue.nl/ws/portalfiles/portal/318595789/1011230_-_Snijders_J.A.C._-_MSc_thesis_Thesis_-_MAP.pdf

Implementing a rotated surface code, as far as I understand it:

1. Initialize qubits
   - Define grid size for a rotated surface code of distance d.
     - This means the number of data qubits (physical non-ancilla qubits) is d^2.
     - 3x3 grid ==> 9 data qubits.

2. Create a lattice:
   - Let each qubit be a vertex in a d×d grid.
   - Assign a qubit to each of the vertices on the grid.

3. Initialize the logical qubit:
   - Apply a **logical \(X\)** and/or **logical \(Z\)** operator to encode the logical qubit (e.g., by applying a chain of qubits across rows and columns).
    - I'm not suuuuper clear on the nitty gritty of this part...but I imagine we can use the same circuits as the toric code paper? Should be fine as long as we keep track of boundary conditions

4. Apply stabilizer measurements (star and plaquette):
   - A star stabilizer acts on a qubit and its neighboring qubits (connected to a vertex).
   - A plaquette stabilizer acts on four qubits in a square arrangement.
   - Apply each stabilizer by measuring the state of neighboring qubits:
     - For each **plaquette stabilizer**:
       - Measure the parity of four qubits that form a square (e.g., top-left, top-right, bottom-left, bottom-right of a plaquette).
     - For each **star stabilizer**:
       - Measure the parity of four qubits connected to a vertex.

5. Syndrome measurement:
   - Measure the stabilizers (parity of qubits) to detect errors.
   - If the syndrome indicates an error (for example, a flipped bit), you can to correct the error by applying the appropriate gate
   - Store the syndrome outcome (e.g., which stabilizers detected an error)
   - Do we want to do this Pauli frame stuff? That would go here ish

6. Error correction (move this to later if we do PF):
   - If the syndrome indicates a flipped bit or phase, apply a correction based on the syndrome information

7. Iterate:
   - Repeat the process for each time step until the logical qubit is read out with high fidelity.

8. Logical qubit measurement:
   - PF corrections go here if we do that.
   - Once error correction is complete, measure the logical qubit to extract the result.

9. Return the result:
   - The final measurement outcome is the state of the logical qubit after the error correction process.

